In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data

# Add parent directory to path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from config import config as cfg

In [ ]:
ID = data.Field()
LABEL = data.Field(lower=True, sequential=False)
STATEMENT = data.Field(batch_first=True)
SUBJECTS, SPEAKER, SPEAKER_TITLE = data.Field(sequential=False)*3


In [8]:
train_set, val_set, test_set = data.TabularDataset.splits(
                                path='/home/shyam/myProjects/fake-news-classifier/datasets/LIAR_PLUS/',
                                train='train2.tsv',
                                validation='val2.tsv',
                                test='test2.tsv',
                                format='tsv',
                                fields=[('id', ID),
                                       ('label', LABEL),
                                       ('statement', STATEMENT),
                                       ('subjects', SUBJECTS),
                                       ('speaker', SPEAKER),
                                       ('speaker_title', SPEAKER_TITLE),
                                       ('state_info', STATE_INFO),
                                       ('party', PARTY),
                                       ('barely_true', BARELY_TRUE),
                                       ('false', FALSE),
                                       ('half_true', HALF_TRUE),
                                       ('mostly_true', MOSTLY_TRUE),
                                       ('pants_on_fire', PANTS_ON_FIRE),
                                       ('context', CONTEXT),
                                       ('justification', JUSTIFICATION)
                                       ]
                               )

In [10]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (train_set, val_set, test_set),
        batch_sizes=(16, 4, 4),
        device=cfg.device
    )

In [11]:
ti = iter(train_iter)

In [12]:
ti

<generator object Iterator.__iter__ at 0x7f6720d651b0>

In [14]:
next(ti)

AttributeError: 'Field' object has no attribute 'vocab'